In [6]:
pip install transformers datasets evaluate

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.9.0
    Uninstalling fsspec-0.9.0:
      Successfully uninstalled fsspec-0.9.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
from datasets import load_dataset

In [4]:
sandra = load_dataset("data")

Found cached dataset json (C:/Users/ariel/.cache/huggingface/datasets/json/data-99bffce22e1cf858/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
data = sandra['train'].train_test_split(test_size=0.15)

In [11]:
data["train"][0]

{'message': "Only if you're not busy and you wanna"}

In [18]:
def preprocess_function(examples):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
    return tokenizer([" ".join(x) for x in examples["message"]])

In [19]:
tokenized = data.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=data["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/3533 [00:00<?, ? examples/s]

TypeError: 'GPT2Tokenizer' object is not callable

# Sandra

In [22]:
def preprocess_function(examples):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

    # Set the padding token explicitly
    tokenizer.pad_token = tokenizer.eos_token

    return tokenizer.batch_encode_plus(
        [x for x in examples["message"]],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
        pad_to_max_length=True
    )

tokenized = data.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=data["train"].column_names,
)


Map (num_proc=4):   0%|          | 0/3533 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/624 [00:00<?, ? examples/s]

In [28]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    block_size = 128
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [29]:
lm_dataset = tokenized.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/3533 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/624 [00:00<?, ? examples/s]

In [35]:
import transformers
print(transformers.__version__)

2.8.0


In [41]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [42]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

ImportError: cannot import name 'TFAutoModelForCausalLM' from 'transformers' (C:\Ari\lib\site-packages\transformers\__init__.py)

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)